In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from datasets import load_dataset
import pandas as pd
import openai
import os
import json

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

dataset = load_dataset("inkoziev/jokes_dialogues")

df = dataset['train'].to_pandas()

selected_jokes = df.sample(n=50)

def convert_to_chat_format(df):
    chat_format = []
    for _, row in df.iterrows():
        chat_format.append({
            "messages": [
                {"role": "system", "content": "You are a humorous assistant."},
                {"role": "user", "content": row['context']},
                {"role": "assistant", "content": row['utterance']}
            ]
        })
    return chat_format

formatted_data = convert_to_chat_format(selected_jokes)

with open("jokes_for_finetuning_chat.jsonl", "w", encoding="utf-8") as f:
    for item in formatted_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

jokes_dialog_dataset.json:   0%|          | 0.00/61.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/114579 [00:00<?, ? examples/s]

In [ ]:
!curl https://api.openai.com/v1/files \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -F purpose="fine-tune" \
  -F file="@/content/jokes_for_finetuning_chat.jsonl"


{
  "object": "file",
  "id": "file-XGXCkaVNrGy1CRVmhVnSxX",
  "purpose": "fine-tune",
  "filename": "jokes_for_finetuning_chat.jsonl",
  "bytes": 26664,
  "created_at": 1734658405,
  "status": "processed",
  "status_details": null
}


In [ ]:
from openai import OpenAI

client = OpenAI()

client.fine_tuning.jobs.create(
    training_file="file-XGXCkaVNrGy1CRVmhVnSxX",
    model="gpt-4o-mini-2024-07-18"
)

FineTuningJob(id='ftjob-I9R05FTJX3pPt4WxFMZAph8j', created_at=1734658415, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-yyyPGCw9VCJjgaycEyhn8xFf', result_files=[], seed=338544673, status='validating_files', trained_tokens=None, training_file='file-XGXCkaVNrGy1CRVmhVnSxX', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'batch_size': 'auto', 'learning_rate_multiplier': 'auto', 'n_epochs': 'auto'}}})

In [ ]:
fine_tuning_job = client.fine_tuning.jobs.retrieve("ftjob-I9R05FTJX3pPt4WxFMZAph8j")
print(fine_tuning_job)

FineTuningJob(id='ftjob-I9R05FTJX3pPt4WxFMZAph8j', created_at=1734658415, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::AgMH5gFH', finished_at=1734658762, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-yyyPGCw9VCJjgaycEyhn8xFf', result_files=['file-LLgycUcJ3DEfpKStj9dGyP'], seed=338544673, status='succeeded', trained_tokens=13170, training_file='file-XGXCkaVNrGy1CRVmhVnSxX', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'n_epochs': 3, 'batch_size': 1, 'learning_rate_multiplier': 1.8}}})


In [7]:
client = openai.OpenAI()

completion = client.chat.completions.create(
    model="ft:gpt-4o-mini-2024-07-18:personal::AgMH5gFH",
    messages=[
        {"role": "system", "content": "Ты анекдотный мастер, продолжи фразу остроумным анекдотом"},
        {"role": "user", "content": "Раз яблоко, два яблоко"}
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Три яблоко, четыре яблоко.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
